In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd 'drive/MyDrive/Colab Notebooks/MT'


/content/drive/MyDrive/Colab Notebooks/MT


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/MT


# Installation

In [ ]:
!pip install datasets transformers sentencepiece sacrebleu==1.5.1

     |████████████████████████████████| 311 kB 5.2 MB/s 
     |████████████████████████████████| 3.4 MB 47.4 MB/s 
     |████████████████████████████████| 1.2 MB 60.3 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 17.1 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 133 kB 49.8 MB/s 
     |████████████████████████████████| 243 kB 68.9 MB/s 
     |████████████████████████████████| 3.3 MB 52.4 MB/s 
     |████████████████████████████████| 895 kB 62.3 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 144 kB 48.2 MB/s 
     |████████████████████████████████| 271 kB 68.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Results

In [ ]:
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16

In [ ]:
results = {}

# Bg -> En -> Mk -> Ro

In [ ]:
from datasets import load_dataset

In [ ]:
bg_ro_test = load_dataset("setimes", "bg-ro", split='train[12500:15000]')

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.6M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset setimes downloaded and prepared to /root/.cache/huggingface/datasets/setimes/bg-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d. Subsequent calls will reuse this data.


In [ ]:
len(bg_ro_test)

2500

In [ ]:
bg_ro_test[:5]

{'id': ['12500', '12501', '12502', '12503', '12504'],
 'translation': [{'bg': 'SETimes: Смятате ли, че този вид проблеми не са вече толкова тежки?',
   'ro': 'SETimes: Consideraţi că lucrurile se îmbunătăţesc?'},
  {'bg': 'Как се промениха през последните няколко години?',
   'ro': 'Cum s-a modificat situaţia în ultimii ani?'},
  {'bg': 'Koптас: Относително погледнато, положението се подобрява, тъй като преди 20 години бе невъзможно да се говори за тези проблеми и турските интелектуалци не знаеха за съществуването на тези проблеми.',
   'ro': 'Koptas: Relativ vorbind, se îmbunătăţesc lucrurile, pentru că acum 20 de ani nu se putea vorbi despre aceste aspecte, iar intelectualii turci nu erau la curent cu aceste fenomene.'},
  {'bg': 'Само арменците защитаваха правата си и кюрдите защитаваха собствените си права, но сега е различно: сега съществува група интелектуалци [от всички етноси], които се противопоставят на този вид дискриминация.',
   'ro': 'Doar armenii îşi apărau propria cauză

In [ ]:
src_text = [sample["translation"]["bg"] for sample in bg_ro_test]
src_text[:5]

['SETimes: Смятате ли, че този вид проблеми не са вече толкова тежки?',
 'Как се промениха през последните няколко години?',
 'Koптас: Относително погледнато, положението се подобрява, тъй като преди 20 години бе невъзможно да се говори за тези проблеми и турските интелектуалци не знаеха за съществуването на тези проблеми.',
 'Само арменците защитаваха правата си и кюрдите защитаваха собствените си права, но сега е различно: сега съществува група интелектуалци [от всички етноси], които се противопоставят на този вид дискриминация.',
 'С тези идеи расте младото поколение, така че хората могат да разберат, че национализмът не е толкова добър, колкото твърди държавата.']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer

bg_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bg-en").to(DEVICE)
bg_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bg-en")

en_mk_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-mk").to(DEVICE)
en_mk_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mk")

mk_ro_model = MarianMTModel.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000").to(DEVICE)
mk_ro_tokenizer = MarianTokenizer.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000")

# Bg -> En
batch_index = 0
en_gen = []
while batch_index < len(src_text):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)

    batch = bg_en_tokenizer(src_text[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = bg_en_model.generate(**batch)
    en_gen.extend(bg_en_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

# En -> Mk
batch_index = 0
mk_gen = []
while batch_index < len(en_gen):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)

    batch = en_mk_tokenizer(en_gen[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = en_mk_model.generate(**batch)
    mk_gen.extend(en_mk_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

# Mk -> Ro
batch_index = 0
ro_gen = []
while batch_index < len(mk_gen):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)
    batch = mk_ro_tokenizer(mk_gen[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = mk_ro_model.generate(**batch)
    ro_gen.extend(mk_ro_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/973k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/979k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400
Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400
Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400


In [ ]:
from datasets import load_metric


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

metric = load_metric("sacrebleu")

predictions, references = postprocess_text(ro_gen, [sample["translation"]["ro"] for sample in bg_ro_test])
bleu_score = metric.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_score['score']:.4f}")

results["bg_en_mk_ro"] = bleu_score

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

BLEU: 23.1548


In [ ]:
ro_gen[:5]

['SETimes: Credeţi că problema actuală nu este atât de gravă?',
 'Ce s-a întâmplat în ultimii câţiva ani?',
 'Koptas: Cu siguranţă, situaţia s-a îmbunătăţit deoarece în urmă cu 20 de ani nu s-a putut ajunge la aceste probleme şi instituţiile turceşti nu şi-au dat seama de prezenţa acestor probleme.',
 'Numai ţările membre şi-au apărat drepturile, dar acum este diferit: acum există grupuri de intelectuali [din toate grupurile etnice] care se opun acestei diminuări.',
 'Aceste idei au pus în scenă marea generaţie de oameni care să înţeleagă că naţionalismul nu este atât de bun ca să-i susţină pe cetăţeni.']

In [ ]:
bg_ro_test[:5]["translation"]

[{'bg': 'SETimes: Смятате ли, че този вид проблеми не са вече толкова тежки?',
  'ro': 'SETimes: Consideraţi că lucrurile se îmbunătăţesc?'},
 {'bg': 'Как се промениха през последните няколко години?',
  'ro': 'Cum s-a modificat situaţia în ultimii ani?'},
 {'bg': 'Koптас: Относително погледнато, положението се подобрява, тъй като преди 20 години бе невъзможно да се говори за тези проблеми и турските интелектуалци не знаеха за съществуването на тези проблеми.',
  'ro': 'Koptas: Relativ vorbind, se îmbunătăţesc lucrurile, pentru că acum 20 de ani nu se putea vorbi despre aceste aspecte, iar intelectualii turci nu erau la curent cu aceste fenomene.'},
 {'bg': 'Само арменците защитаваха правата си и кюрдите защитаваха собствените си права, но сега е различно: сега съществува група интелектуалци [от всички етноси], които се противопоставят на този вид дискриминация.',
  'ro': 'Doar armenii îşi apărau propria cauză, iar kurzii pe a lor. Acum situaţiile se întrepătrund: acum există un grup c

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer

bg_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bg-en").to(DEVICE)
bg_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bg-en")

en_mk_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-mk").to(DEVICE)
en_mk_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mk")

mk_ro_model = MarianMTModel.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000").to(DEVICE)
mk_ro_tokenizer = MarianTokenizer.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000")

bg_samples = [
    "Първото учение включваше противопожарни действия, спасителни операции и евакуация на ранени хора от втория етаж на сграда.",
    "За минути бе монтирано оборудване за изнасяне на ранените, специални въжета и специален дюшек.",
    "След това бе направена симулация на взривяване и унищожаване на мост.",
    "Спасителите извадиха хората от разбитите автомобили и помогнаха да се отстранят токсичните химикали.",
    "Ранените бяха транспортирани с хеликоптер до лазарет, в който имаше операционна, оборудвана с необходимата апаратура за действия в случай на природни, технологични и биологични инциденти."
]

# Bg -> En
batch = bg_en_tokenizer(bg_samples, return_tensors="pt", padding=True).to(DEVICE)
gen = bg_en_model.generate(**batch)
en_gen = bg_en_tokenizer.batch_decode(gen, skip_special_tokens=True)

# En -> Mk
batch = en_mk_tokenizer(en_gen, return_tensors="pt", padding=True).to(DEVICE)
gen = en_mk_model.generate(**batch)
mk_gen = en_mk_tokenizer.batch_decode(gen, skip_special_tokens=True)


# Mk -> Ro
batch = mk_ro_tokenizer(mk_gen, return_tensors="pt", padding=True).to(DEVICE)
gen = mk_ro_model.generate(**batch)
ro_gen = mk_ro_tokenizer.batch_decode(gen, skip_special_tokens=True)

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/973k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/292M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/979k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

In [ ]:
ro_gen

['Prima declaraţie a inclus operaţiuni pro-afaceri, operaţiuni speciale şi acţiuni ale unor persoane mai importante decât oraşul.',
 'Pentru câteva minute a fost prezentată o operaţiune de deminare a şcolilor, unei şcoli speciale şi unei mese rotunde.',
 'După aceasta, situaţia s-a îmbunătăţit şi s-a îmbunătăţit într-o singură locaţie.',
 'Suspecţii l-au identificat pe cel al autorilor săi şi l-au lăsat să-i elimine pe naţionaliştii turci.',
 'Războaiele au fost transportate cu elicopterul într-o zonă care a avut parte de o atmosferă operaţională însoţită de o operaţiune ilegală de acţiuni în cazuri de corupţie, tehnologii şi instituţii de securitate.']

In [ ]:
true_ro_preds = [
    "Primul exerciţiu a presupus combaterea unui incendiu, operaţiuni de salvare şi evacuarea victimelor de la al doilea etaj al unei clădiri.",
    "Echipamentul pentru coborârea victimelor, frânghii speciale şi o pernă specială au fost asamblate în câteva minute.",
    "După aceasta a fost simulată explozia şi distrugerea unui pod.",
    "Salvatorii au scos oameni din vehicule zdrobite şi au ajutat la eliminarea substanţelor chimice toxice.",
    "Cei răniţi au fost transportaţi cu elicopterul la un spital improvizat, dotat cu o sală de operaţii echipată pentru a face faţă incidentelor naturale, tehnologice şi biologice."
]

# Sr -> En -> Mk -> Ro

In [ ]:
from datasets import load_dataset

sr_ro_test = load_dataset("setimes", "ro-sr", split='train[12500:15000]')

sr_ro_test[:5]

Downloading:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset setimes downloaded and prepared to /root/.cache/huggingface/datasets/setimes/ro-sr/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d. Subsequent calls will reuse this data.


{'id': ['12500', '12501', '12502', '12503', '12504'],
 'translation': [{'ro': 'Lemierres a afirmat că BERD va depune eforturi pentru crearea unui climat bun pentru investiţii, pentru încheierea Acordurilor de Comerţ Liber şi pentru a se asigura că întreprinderile mici şi mijlocii au acces mai uşor la împrumuturi în Europa de Sud-est.',
   'sr': 'Lemijer je rekao da će EBRD pokušati da uspostavi dobru investicionu klimu, zaključi Sporazume o slobodnoj trgovini i obezbedi lakši pristup kreditima u jugoistočnoj Evropi za mala i srednja preduzeća.'},
  {'ro': 'Mişcarea de nealiniere are încă influenţă, afirmă liderii',
   'sr': 'Pokret nesvrstanih i dalje uticajan, kažu lideri'},
  {'ro': 'Belgradul găzduieşte conferinţa aniversară a unei iniţiative co-lansate de fostul preşedinte Tito.',
   'sr': 'Beograd je domaćin konferencije na godišnjicu osnivanja pokreta, čiji je jedan od osnivača bio pokojni Tito.'},
  {'ro': 'De Biljana Pekusic, din Belgrad pentru Southeast European Times -- 07/09

In [ ]:
sr_src_text = [sample["translation"]["sr"] for sample in sr_ro_test]
sr_src_text[:5]

['Lemijer je rekao da će EBRD pokušati da uspostavi dobru investicionu klimu, zaključi Sporazume o slobodnoj trgovini i obezbedi lakši pristup kreditima u jugoistočnoj Evropi za mala i srednja preduzeća.',
 'Pokret nesvrstanih i dalje uticajan, kažu lideri',
 'Beograd je domaćin konferencije na godišnjicu osnivanja pokreta, čiji je jedan od osnivača bio pokojni Tito.',
 'Biljana Pekušić za Southeast European Times iz Beograda -- 7.9.2011.',
 'Ministri inostranih poslova i šefovi delegacija zemalja Pokreta nesvrstanih poziraju ispred zgrade Skupštine Srbije u Beogradu 5. septembra. [Rojters]']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer

sr_en_model = MarianMTModel.from_pretrained("opus-mt-uk-en-finetuned-sr-to-en/checkpoint-12500").to(DEVICE)
sr_en_tokenizer = MarianTokenizer.from_pretrained("opus-mt-uk-en-finetuned-sr-to-en/checkpoint-12500")

en_mk_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-mk").to(DEVICE)
en_mk_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mk")

mk_ro_model = MarianMTModel.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000").to(DEVICE)
mk_ro_tokenizer = MarianTokenizer.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000")

# Sr -> En
batch_index = 0
en_gen = []
while batch_index < len(sr_src_text):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)

    batch = sr_en_tokenizer(sr_src_text[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = sr_en_model.generate(**batch)
    en_gen.extend(sr_en_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

# En -> Mk
batch_index = 0
mk_gen = []
while batch_index < len(en_gen):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)

    batch = en_mk_tokenizer(en_gen[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = en_mk_model.generate(**batch)
    mk_gen.extend(en_mk_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

# Mk -> Ro
batch_index = 0
ro_gen = []
while batch_index < len(mk_gen):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)
    batch = mk_ro_tokenizer(mk_gen[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = mk_ro_model.generate(**batch)
    ro_gen.extend(mk_ro_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400
Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400
Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400


In [ ]:
from datasets import load_metric


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

metric = load_metric("sacrebleu")

predictions, references = postprocess_text(ro_gen, [sample["translation"]["ro"] for sample in sr_ro_test])
bleu_score = metric.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_score['score']:.4f}")

results["sr_en_mk_ro"] = bleu_score

BLEU: 20.8195


In [ ]:
ro_gen[:5]

['Lemir a declarat că BERD va încerca să obţină un climat de investiţie bun, să încheie acorduri de comerţ liber şi să ofere un sprijin mai bun ţărilor din Europa de Sud-est pentru întreprinderile mici şi mijlocii.',
 'Rămăşiţa experţilor şi încă a celorlalţi imprejurimi, declară liderii.',
 'Belgradul a fost găzduit de conferinţa de anul trecut de către oficialii de la Tiro.',
 'De Biljana Pekusic, din Belgrad pentru Southeast European Times -- 7/09/11',
 'Miniştrii de externe şi şefii de delegaţie ai ţărilor din cadrul Preşedinţiilor de la Poorish pentru activităţi interne cer construcţia unei comunităţi sârbe la Belgrad în 5 septembrie. [Reuters]']

In [ ]:
sr_ro_test[:5]["translation"]

[{'ro': 'Lemierres a afirmat că BERD va depune eforturi pentru crearea unui climat bun pentru investiţii, pentru încheierea Acordurilor de Comerţ Liber şi pentru a se asigura că întreprinderile mici şi mijlocii au acces mai uşor la împrumuturi în Europa de Sud-est.',
  'sr': 'Lemijer je rekao da će EBRD pokušati da uspostavi dobru investicionu klimu, zaključi Sporazume o slobodnoj trgovini i obezbedi lakši pristup kreditima u jugoistočnoj Evropi za mala i srednja preduzeća.'},
 {'ro': 'Mişcarea de nealiniere are încă influenţă, afirmă liderii',
  'sr': 'Pokret nesvrstanih i dalje uticajan, kažu lideri'},
 {'ro': 'Belgradul găzduieşte conferinţa aniversară a unei iniţiative co-lansate de fostul preşedinte Tito.',
  'sr': 'Beograd je domaćin konferencije na godišnjicu osnivanja pokreta, čiji je jedan od osnivača bio pokojni Tito.'},
 {'ro': 'De Biljana Pekusic, din Belgrad pentru Southeast European Times -- 07/09/11',
  'sr': 'Biljana Pekušić za Southeast European Times iz Beograda -- 7.

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer

sr_en_model = MarianMTModel.from_pretrained("opus-mt-uk-en-finetuned-sr-to-en/checkpoint-12500").to(DEVICE)
sr_en_tokenizer = MarianTokenizer.from_pretrained("opus-mt-uk-en-finetuned-sr-to-en/checkpoint-12500")

en_mk_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-mk").to(DEVICE)
en_mk_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mk")

mk_ro_model = MarianMTModel.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000").to(DEVICE)
mk_ro_tokenizer = MarianTokenizer.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000")

sr_samples = [
    'Videanuov azerbejdžanski kolega nazvao je potpisivanje memoranduma "istorijskim događajem".'
]

# Sr -> En
batch = sr_en_tokenizer(sr_samples, return_tensors="pt", padding=True).to(DEVICE)
gen = sr_en_model.generate(**batch)
en_gen = sr_en_tokenizer.batch_decode(gen, skip_special_tokens=True)

# En -> Mk
batch = en_mk_tokenizer(en_gen, return_tensors="pt", padding=True).to(DEVICE)
gen = en_mk_model.generate(**batch)
mk_gen = en_mk_tokenizer.batch_decode(gen, skip_special_tokens=True)

# Mk -> Ro
batch = mk_ro_tokenizer(mk_gen, return_tensors="pt", padding=True).to(DEVICE)
gen = mk_ro_model.generate(**batch)
ro_gen = mk_ro_tokenizer.batch_decode(gen, skip_special_tokens=True)

In [16]:
ro_gen

['Colegul Azerbejjanski de la Videanu a făcut apel la "insultarea tradiţiilor istorice".']

In [ ]:
true_ro_pred = [
    'Omologul azer al lui Videanu a numit semnarea memorandumului drept "un eveniment istoric".'
]

# Grk -> En -> Mk -> Ro

In [ ]:
from datasets import load_dataset

grk_ro_test = load_dataset("setimes", "el-ro", split='train[12500:15000]')

grk_ro_test[:5]

Downloading:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset setimes downloaded and prepared to /root/.cache/huggingface/datasets/setimes/el-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d. Subsequent calls will reuse this data.


{'id': ['12500', '12501', '12502', '12503', '12504'],
 'translation': [{'el': 'Η ανεργία έφτασε το 11,9% τον Απρίλιο, σύμφωνα με το ΑΝΑ, και η πρόβλεψη του ΔΝΤ είναι ότι θα ανέλθει σε περίπου 15% το 2012.',
   'ro': 'Rata şomajului a ajuns în luna aprilie la 11,9%, potrivit ANA, iar previziunile FMI afirmă că va atinge un maxim de aproape 15%, în 2012.'},
  {'el': 'Υπάρχει επίσης ανησυχία για την ικανότητα της κυβέρνησης να διατηρήσει τη νέα πολιτική.',
   'ro': 'De asemenea, există îngrijorări că guvernul nu va avea abilitatea de a-şi menţine noua politică.'},
  {'el': '"Το πραγματικό πρόβλημα...είναι η αξιοπιστία του πολιτικού συστήματος", ανέφερε ο Γλυνός.',
   'ro': '„Adevărata problemă... este credibilitatea sistemului politic”, a afirmat Glynos.'},
  {'el': 'Ο Παπανδρέου έχει κάνει στροφή 180 μοιρών από το σοσιαλιστικό πρόγραμμα του κόμματός του, ΠΑΣΟΚ, εξοργίζοντας κάποια μέλη.',
   'ro': 'Papandreou s-a întors la 180 de grade faţă de platforma socialistă a partidului său PASOK,

In [ ]:
grk_src_text = [sample["translation"]["el"] for sample in grk_ro_test]
grk_src_text[:5]

['Η ανεργία έφτασε το 11,9% τον Απρίλιο, σύμφωνα με το ΑΝΑ, και η πρόβλεψη του ΔΝΤ είναι ότι θα ανέλθει σε περίπου 15% το 2012.',
 'Υπάρχει επίσης ανησυχία για την ικανότητα της κυβέρνησης να διατηρήσει τη νέα πολιτική.',
 '"Το πραγματικό πρόβλημα...είναι η αξιοπιστία του πολιτικού συστήματος", ανέφερε ο Γλυνός.',
 'Ο Παπανδρέου έχει κάνει στροφή 180 μοιρών από το σοσιαλιστικό πρόγραμμα του κόμματός του, ΠΑΣΟΚ, εξοργίζοντας κάποια μέλη.',
 '"Όλα αυτά [τα μέτρα λιτότητας] συγκρούονται με όσα είχε πει κατά τη διάρκεια της προεκλογικής εκστρατείας του...αλλά αποδείχθηκε ευέλικτος", ανέφερε ο Στουρνάρας.']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer

grk_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-grk-en").to(DEVICE)
grk_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-grk-en")

en_mk_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-mk").to(DEVICE)
en_mk_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mk")

mk_ro_model = MarianMTModel.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000").to(DEVICE)
mk_ro_tokenizer = MarianTokenizer.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000")

# Grk -> En
batch_index = 0
en_gen = []
while batch_index < len(grk_src_text):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)

    batch = grk_en_tokenizer(grk_src_text[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = grk_en_model.generate(**batch)
    en_gen.extend(grk_en_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

# En -> Mk
batch_index = 0
mk_gen = []
while batch_index < len(en_gen):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)

    batch = en_mk_tokenizer(en_gen[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = en_mk_model.generate(**batch)
    mk_gen.extend(en_mk_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

# Mk -> Ro
batch_index = 0
ro_gen = []
while batch_index < len(mk_gen):
    if batch_index % 100 == 0:
        print("Batch index", batch_index)
    batch = mk_ro_tokenizer(mk_gen[batch_index:batch_index + BATCH_SIZE], return_tensors="pt", padding=True).to(DEVICE)
    gen = mk_ro_model.generate(**batch)
    ro_gen.extend(mk_ro_tokenizer.batch_decode(gen, skip_special_tokens=True))

    batch_index += BATCH_SIZE

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400
Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400
Batch index 0
Batch index 400
Batch index 800
Batch index 1200
Batch index 1600
Batch index 2000
Batch index 2400


In [ ]:
from datasets import load_metric


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

metric = load_metric("sacrebleu")

predictions, references = postprocess_text(ro_gen, [sample["translation"]["ro"] for sample in grk_ro_test])
bleu_score = metric.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_score['score']:.4f}")

results["grk_en_mk_ro"] = bleu_score

BLEU: 23.8915


In [ ]:
ro_gen[:5]

['Rata şomajului a ajuns la 11,9% în aprilie, potrivit estimărilor FMI, şi s-a ridicat la aproximativ 15% în 2012.',
 'De asemenea, există temeri cu privire la capacitatea guvernului de a menţine o politică nouă.',
 '"Problema de astăzi... este credibilitatea sistemului politic," a afirmat Gruin.',
 'Papandreou a cheltuit 180 de locuri din programul socialist al partidului său, PASOK, din cauza câtorva membri.',
 '"Toate aceste [măsuri] sunt în conformitate cu ceea ce a descris în timpul campaniei electorale a ţării sale... dar aceasta s-a dovedit a fi complet diferită", a afirmat Strugaras.']

In [ ]:
grk_ro_test[:5]["translation"]

[{'el': 'Η ανεργία έφτασε το 11,9% τον Απρίλιο, σύμφωνα με το ΑΝΑ, και η πρόβλεψη του ΔΝΤ είναι ότι θα ανέλθει σε περίπου 15% το 2012.',
  'ro': 'Rata şomajului a ajuns în luna aprilie la 11,9%, potrivit ANA, iar previziunile FMI afirmă că va atinge un maxim de aproape 15%, în 2012.'},
 {'el': 'Υπάρχει επίσης ανησυχία για την ικανότητα της κυβέρνησης να διατηρήσει τη νέα πολιτική.',
  'ro': 'De asemenea, există îngrijorări că guvernul nu va avea abilitatea de a-şi menţine noua politică.'},
 {'el': '"Το πραγματικό πρόβλημα...είναι η αξιοπιστία του πολιτικού συστήματος", ανέφερε ο Γλυνός.',
  'ro': '„Adevărata problemă... este credibilitatea sistemului politic”, a afirmat Glynos.'},
 {'el': 'Ο Παπανδρέου έχει κάνει στροφή 180 μοιρών από το σοσιαλιστικό πρόγραμμα του κόμματός του, ΠΑΣΟΚ, εξοργίζοντας κάποια μέλη.',
  'ro': 'Papandreou s-a întors la 180 de grade faţă de platforma socialistă a partidului său PASOK, ceea ce i-a deranjat pe unii membri.'},
 {'el': '"Όλα αυτά [τα μέτρα λιτότητ

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer

grk_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-grk-en").to(DEVICE)
grk_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-grk-en")

en_mk_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-mk").to(DEVICE)
en_mk_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mk")

mk_ro_model = MarianMTModel.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000").to(DEVICE)
mk_ro_tokenizer = MarianTokenizer.from_pretrained("opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000")

gr_samples = [
    "Έχουμε επίσης παρατηρήσει έλλειψη συσχετισμού μεταξύ χρηματοδοτούμενων προγραμμάτων ακόμα και μέσα σε μια χώρα."
]

# Grk -> En
batch = grk_en_tokenizer(gr_samples, return_tensors="pt", padding=True).to(DEVICE)
gen = grk_en_model.generate(**batch)
en_gen = grk_en_tokenizer.batch_decode(gen, skip_special_tokens=True)

# En -> Mk
batch = en_mk_tokenizer(en_gen, return_tensors="pt", padding=True).to(DEVICE)
gen = en_mk_model.generate(**batch)
mk_gen = en_mk_tokenizer.batch_decode(gen, skip_special_tokens=True)

# Mk -> Ro
batch = mk_ro_tokenizer(mk_gen, return_tensors="pt", padding=True).to(DEVICE)
gen = mk_ro_model.generate(**batch)
ro_gen = mk_ro_tokenizer.batch_decode(gen, skip_special_tokens=True)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

In [18]:
ro_gen

['De asemenea, am văzut lipsa unei comisii printre programele financiare şi în cadrul unei ţări.']

In [19]:
true_ro_pred = [
    "Am remarcat de asemenea lipsa corelării dintre proiectele finanţate chiar şi în acelaşi judeţ."
]

# Comparison

In [ ]:
results

{'bg_en_mk_ro': {'bp': 0.9415376297102508,
  'counts': [51, 6, 1, 0],
  'precisions': [12.289156626506024,
   1.518987341772152,
   0.26666666666666666,
   0.14084507042253522],
  'ref_len': 440,
  'score': 0.8615576365799047,
  'sys_len': 415,
  'totals': [415, 395, 375, 355]},
 'grk_en_mk_ro': {'bp': 0.9664639209783824,
  'counts': [133, 52, 35, 28],
  'precisions': [23.877917414721722,
   9.683426443202979,
   6.769825918762089,
   5.633802816901408],
  'ref_len': 576,
  'score': 9.365625934028103,
  'sys_len': 557,
  'totals': [557, 537, 517, 497]},
 'sr_en_mk_ro': {'bp': 1.0,
  'counts': [74, 10, 2, 0],
  'precisions': [12.585034013605442,
   1.7605633802816902,
   0.36496350364963503,
   0.0946969696969697],
  'ref_len': 508,
  'score': 0.9354549862142115,
  'sys_len': 588,
  'totals': [588, 568, 548, 528]}}

In [ ]:
results

{'bg_en_mk_ro': {'bp': 1.0,
  'counts': [8394, 1553, 742, 504],
  'precisions': [13.72667658746382,
   2.6478661915397863,
   1.321436839949422,
   0.9393171313553005],
  'ref_len': 60233,
  'score': 2.591671914130915,
  'sys_len': 61151,
  'totals': [61151, 58651, 56151, 53656]},
 'grk_en_mk_ro': {'bp': 0.9621546611925638,
  'counts': [9322, 2458, 1380, 988],
  'precisions': [15.200482658535392,
   4.1783534771448485,
   2.449979583503471,
   1.8354759604666717],
  'ref_len': 63693,
  'score': 3.955380959325976,
  'sys_len': 61327,
  'totals': [61327, 58827, 56327, 53828]},
 'sr_en_mk_ro': {'bp': 1.0,
  'counts': [9444, 2029, 1019, 695],
  'precisions': [15.367842090703464,
   3.4417247637948876,
   1.8050413618408234,
   1.2881343366571525],
  'ref_len': 57644,
  'score': 3.330115796403364,
  'sys_len': 61453,
  'totals': [61453, 58953, 56453, 53954]}}

In [ ]:
results